In [6]:
import cv2
import numpy as np
import time
import csv

# -----------------------------
# PARAMETERS
# -----------------------------
SNAP_INTERVAL = 5        # seconds between snapshots
CSV_FILE = "bubble_log.csv"
MAX_RUNS = 12            # number of snapshots (12 × 5s = 1 minute test)
AREA_THRESHOLD = 500     # minimum pixel area for valid bubble

# -----------------------------
# FUNCTIONS
# -----------------------------
def bubble_features(frame):
    """Return quantitative features of the bubble if present, else None."""
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Blue color range (adjust if needed)
    lower_blue = np.array([100, 150, 50])
    upper_blue = np.array([140, 255, 255])
    mask = cv2.inRange(hsv, lower_blue, upper_blue)

    # Find contours (blobs of blue)
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not contours:
        return None

    # Take largest blue contour (assume that's the bubble)
    cnt = max(contours, key=cv2.contourArea)
    area = cv2.contourArea(cnt)
    if area < AREA_THRESHOLD:
        return None

    # Bounding box for position
    x, y, w, h = cv2.boundingRect(cnt)
    cx, cy = x + w//2, y + h//2

    # Mask only inside this contour
    bubble_mask = np.zeros_like(mask)
    cv2.drawContours(bubble_mask, [cnt], -1, 255, -1)

    # Compute mean color inside bubble
    mean_bgr = cv2.mean(frame, mask=bubble_mask)[:3]
    mean_hsv = cv2.mean(hsv, mask=bubble_mask)[:3]

    return {
        "area": int(area),
        "center_x": cx,
        "center_y": cy,
        "mean_B": round(mean_bgr[0], 1),
        "mean_G": round(mean_bgr[1], 1),
        "mean_R": round(mean_bgr[2], 1),
        "mean_H": round(mean_hsv[0], 1),
        "mean_S": round(mean_hsv[1], 1),
        "mean_V": round(mean_hsv[2], 1),
    }

# -----------------------------
# MAIN LOOP
# -----------------------------
cap = cv2.VideoCapture(0)  # try 1 or 2 if 0 doesn't work
if not cap.isOpened():
    raise RuntimeError("❌ Could not open webcam. Try another index (0,1,2).")

time_series = []

try:
    for i in range(MAX_RUNS):   # limited runs for testing
        ret, frame = cap.read()
        if not ret or frame is None:
            print("⚠️ Empty frame, skipping...")
            time.sleep(SNAP_INTERVAL)
            continue

        current_time = time.time()
        features = bubble_features(frame)

        if features:
            row = [
                current_time, 1, features["area"], features["center_x"], features["center_y"],
                features["mean_R"], features["mean_G"], features["mean_B"],
                features["mean_H"], features["mean_S"], features["mean_V"]
            ]
            print(f"Snapshot {i+1}/{MAX_RUNS} | Bubble Detected | Area={features['area']} | "
                  f"RGB=({features['mean_R']},{features['mean_G']},{features['mean_B']})")
        else:
            row = [current_time, 0, 0, None, None, None, None, None, None, None, None]
            print(f"Snapshot {i+1}/{MAX_RUNS} | No bubble detected")

        time_series.append(row)
        time.sleep(SNAP_INTERVAL)

finally:
    cap.release()

# -----------------------------
# SAVE CSV
# -----------------------------
with open(CSV_FILE, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow([
        "timestamp", "bubble_present", "area", "center_x", "center_y",
        "mean_R", "mean_G", "mean_B", "mean_H", "mean_S", "mean_V"
    ])
    writer.writerows(time_series)

print(f"\n✅ Data saved to {CSV_FILE}")

Snapshot 1/12 | No bubble detected
Snapshot 2/12 | No bubble detected
Snapshot 3/12 | No bubble detected
Snapshot 4/12 | No bubble detected
Snapshot 5/12 | No bubble detected
Snapshot 6/12 | Bubble Detected | Area=763 | RGB=(32.8,105.6,154.8)
Snapshot 7/12 | Bubble Detected | Area=1870 | RGB=(27.1,96.4,146.2)
Snapshot 8/12 | Bubble Detected | Area=2056 | RGB=(30.1,103.8,156.1)
Snapshot 9/12 | Bubble Detected | Area=771 | RGB=(35.3,111.1,166.2)
Snapshot 10/12 | No bubble detected
Snapshot 11/12 | No bubble detected
Snapshot 12/12 | No bubble detected

✅ Data saved to bubble_log.csv
